In [1]:
import pandas as pd
from keras import Input
from matplotlib import pyplot as plt

from utils.IndicatorGenerator import IndicatorGenerator
from utils.StockDataManager import StockDataManager

In [2]:
stockDataManager = StockDataManager()

df = stockDataManager.history(ticker="MSFT", period="1d", interval="1d")

In [3]:
df.head()

In [4]:
df['daily_return'] = df['adj_close'].pct_change()

In [5]:
df.head()

In [6]:
df['daily_return'] = df['daily_return'].fillna(0)
df.head()

In [7]:
indicatorGen = IndicatorGenerator(df)
windows = [5, 10, 15, 20, 50, 100, 200]

for window in windows:
    sma = indicatorGen.moving_average(window)
    ema = indicatorGen.exponential_moving_average(window)
    df['EMA_' + str(window)] = ema
    df['SMA_' + str(window)] = sma

df.bfill(inplace=True)
macd, signal = indicatorGen.macd()
df['macd'] = macd
df['signal'] = signal

sma, upper_band, lower_band = indicatorGen.bollinger_bands()
df['bb_upper'] = upper_band
df['bb_lower'] = lower_band

df['rsi'] = indicatorGen.rsi()
df['stochastic_oscillator'] = indicatorGen.stochastic_oscillator()
df['cci'] = indicatorGen.cci()

df.bfill(inplace=True)



In [8]:
df['target'] = df['daily_return'].shift(-1)

In [9]:
df.head()

In [10]:
from sklearn.preprocessing import StandardScaler

train_len = int(len(df) * 0.8)
df_train = df[:train_len]
df_test = df[train_len:]

columns = df.columns
scaler = StandardScaler()
scaler.fit(df_train)
df_train[columns] = scaler.transform(df_train)
df_test[columns] = scaler.transform(df_test)


In [11]:
import numpy as np


def create_sequences(sequence_length, data):
    features = data.columns
    target = 'adj_close'
    X = []
    y = []
    x_raw = data[features].values
    y_raw = data[target].values
    for i in range(x_raw.shape[0] - sequence_length):
        X.append(x_raw[i:i + sequence_length])
        y.append(y_raw[i + sequence_length])
    return np.array(X), np.array(y)

In [12]:
X_train, y_train = create_sequences(60, df_train)
X_test, y_test = create_sequences(60, df_test)

In [13]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[1])

In [14]:
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Conv1D, MaxPooling1D, Flatten, concatenate


def build_model():
    input_layer = Input(shape=(60, df_train.shape[1]))

    # LSTM
    lstm1 = LSTM(units=50, return_sequences=True)(input_layer)
    lstm2 = LSTM(units=50, return_sequences=False)(lstm1)

    # CNN
    cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(input_layer)
    max_pool = MaxPooling1D(pool_size=2)(cnn1)
    flatten = Flatten()(max_pool)

    combined = concatenate([lstm2, flatten])
    dense1 = Dense(units=64, activation='relu')(combined)
    dropout = Dropout(0.5)(dense1)
    output = Dense(1)(dropout)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='mse', optimizer='adam')

    model.summary()

    return model

In [15]:
model = build_model()

In [16]:
model.fit(X_train, y_train, epochs=100, batch_size=512, validation_data=(X_test, y_test))

In [17]:
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

In [19]:
import matplotlib.pyplot as plt
plt.plot(train_predictions)
plt.plot(y_train)

In [20]:
plt.plot(test_predictions)
plt.plot(y_test)